In [1]:
import keras
from keras.applications import VGG16
from tensorflow import keras  
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D 
from keras import regularizers  
from keras.layers import Dropout, Rescaling
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import os
from PIL import Image
from tensorflow.keras.callbacks import EarlyStopping ,ModelCheckpoint


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
path = "/kaggle/input/electronics/images"
batch_size = 32
image_size = (224, 224)
train_data = tf.keras.utils.image_dataset_from_directory(path,
                                                     labels = 'inferred' , 
                                                    label_mode = 'categorical', 
                                                           #color_mode="grayscale", 
                                                          validation_split = 0.2,
                                                          subset= 'validation',
                                                         image_size = (image_size),
                                                         seed = 40,
                                                         # normalization=1/255.,
                                                  
                                                   batch_size = batch_size)

val_data = tf.keras.utils.image_dataset_from_directory(path,
                                                     labels = 'inferred' , 
                                                    label_mode = 'categorical',
                                                        # color_mode="grayscale",
                                                         # normalization=1/255.,
                                                       subset= 'validation',
                                                        image_size = (image_size),
                                                       validation_split = 0.2,
                                                       seed = 400,
                                                  
                                                   batch_size = batch_size)
                                                           
    


Found 21980 files belonging to 37 classes.
Using 4396 files for validation.
Found 21980 files belonging to 37 classes.
Using 4396 files for validation.


In [4]:
##saving the class anmes 
import json
class_names = train_data.class_names

with open('/kaggle/working/LATEST_class_names.json', 'w') as f:
    json.dump(class_names, f)

In [9]:
!ls
!pwd

LATEST_class_names.json
/kaggle/working


In [11]:
##load class_names
class_names_path = '/kaggle/working/LATEST_class_names.json'
with open(class_names_path) as f: 
    class_names = json.load(f)

In [12]:

#checkpoint_path = '/home/brian/machine/CIRCUIT PROJECT//step_step_model/updating_model.h5'
#checkpoint_dir = os.path.dirname(checkpoint_path)
check_path = 'kaggle/working/second'
checkpoint = ModelCheckpoint(
    check_path +'best_soo_farmodel.{epoch:02d}.h5',  
    monitor='val_accuracy',
    save_weights_only=True,
    save_best_only=True,  
    verbose=1,
    custom_objects={'class_names': class_names}
)



In [13]:
num_classes = 37
input_shape = (224, 224, 3)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(input_shape))

# Freeze the layers of the base model to avoid updating their weights during training
base_model.trainable = False

# Create your new model on top of the base model
x = base_model.output

# Add your new layers
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)  # Adding Dropout layer with rate 0.5 (50% of neurons will be deactivated during training)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x) 
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

# Combine the base model and the new layers to create the new_model
new_model = tf.keras.Model(inputs=base_model.input, outputs=output)

# Compile the new model with an optimizer, a loss function, and evaluation metrics
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
, loss='categorical_crossentropy', metrics=['accuracy'])

# Now you can see the summary of the new model to understand its architecture
new_model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [15]:
num_epochs = int(len(train_data)/batch_size)
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Fit model
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy' , metrics = ['accuracy'])
history = new_model.fit(
     train_data,
    validation_data = val_data,
    epochs=100,
    callbacks = [checkpoint , early_stopping])

Epoch 1/100
138/138 [==============================] - ETA: 0s - loss: 8.1812 - accuracy: 0.4252
Epoch 1: val_accuracy improved from -inf to 0.50250, saving model to kaggle/working/secondbest_soo_farmodel.01.h5
138/138 [==============================] - 23s 168ms/step - loss: 8.1812 - accuracy: 0.4252 - val_loss: 5.5376 - val_accuracy: 0.5025
Epoch 2/100
137/138 [============================>.] - ETA: 0s - loss: 5.2716 - accuracy: 0.4811
Epoch 2: val_accuracy improved from 0.50250 to 0.50591, saving model to kaggle/working/secondbest_soo_farmodel.02.h5
138/138 [==============================] - 20s 145ms/step - loss: 5.2695 - accuracy: 0.4809 - val_loss: 4.4250 - val_accuracy: 0.5059
Epoch 3/100
137/138 [============================>.] - ETA: 0s - loss: 4.5047 - accuracy: 0.4818
Epoch 3: val_accuracy improved from 0.50591 to 0.50614, saving model to kaggle/working/secondbest_soo_farmodel.03.h5
138/138 [==============================] - 20s 142ms/step - loss: 4.5017 - accuracy: 0.4823 -

In [21]:
!ls
!pwd

LATEST_class_names.json  kaggle
/kaggle/working


In [20]:
!cd kaggle
!ls -la

total 20
drwxr-xr-x 4 root root 4096 Oct 14 06:11 .
drwxr-xr-x 5 root root 4096 Oct 14 06:05 ..
drwxr-xr-x 2 root root 4096 Oct 14 06:05 .virtual_documents
-rw-r--r-- 1 root root  627 Oct 14 06:08 LATEST_class_names.json
drwxr-xr-x 3 root root 4096 Oct 14 06:11 kaggle


In [18]:
!ls

LATEST_class_names.json  kaggle
